### Step 0: Preliminaries
Necessary package installations and imports.

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

### Step 1: Create a simple graph
Create a simple graph with three nodes and two edges

In [ ]:
import redis
from redis.commands.graph import Graph
from redis.commands.graph.node import Node
from redis.commands.graph.edge import Edge

connection = redis.Redis('localhost', 6379, 0)
graph = Graph(connection, 'pyg')

jane = Node(label='person', properties={'age':32})
graph.add_node(jane)
john = Node(label='person', properties={'age':38})
graph.add_node(john)
japan = Node(label='country', properties={'name': 'Japan'})
graph.add_node(japan)
edge1 = Edge(jane, 'visited', japan, properties={'cost': 1.3})
graph.add_edge(edge1)
edge2 = Edge(john, 'visited', japan, properties={'cost': 2.4})
graph.add_edge(edge2)
graph.commit()




Test connection to RedisGraph

In [ ]:
import rg_pyg.connection
import rg_pyg.database

db = rg_pyg.database.Database('localhost', 6379)
conn = rg_pyg.connection.Connection('localhost', 6379, 0)

# Read nodes
person_nodes = conn.execute("MATCH (a:person) RETURN a LIMIT 5").get_as_result_set()
print(person_nodes[0][0])
print(person_nodes[1][0])

# Read edges
edges = conn.execute("MATCH (a:person)-[r]-() RETURN r LIMIT 5").get_as_result_set()
print(edges[0][0])
print(edges[1][0])

# Read paths
edges = conn.execute("MATCH p=(a:person)-[r]-() RETURN p LIMIT 5").get_as_result_set()
print(edges[0][0])
print(edges[1][0])

Export data as torch_geometric

In [ ]:
# Read nodes
# person_nodes = conn.execute("MATCH (a:person) RETURN a LIMIT 5").get_as_torch_geometric()

# Read graph and export it as torch_geometric
query = "MATCH (a:person)-[r]->(b) RETURN a, b, r"
graph_torch = conn.execute(query).get_as_torch_geometric()
print(graph_torch)


In [ ]:

feature_store, graph_store = db.get_torch_geometric_remote_backend()

In [ ]:
feature_store['person', 'age', 32]


In [ ]:
graph_store[('person', 'country'), 'coo']